In [385]:
Pkg.add("StatsBase")
Pkg.add("Distributions")
using StatsBase
using Random
using Distributions
using LinearAlgebra

  Resolving package versions...
   Updating `C:\Users\Bastiiii\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\Bastiiii\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Bastiiii\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\Bastiiii\.julia\environments\v1.4\Manifest.toml`
 [no changes]


In [386]:
"""
ForSample function https://github.com/JuliaStats/StatsBase.jl/blob/08d4b77a4b42ef8cadb67b98da65e7fbd9959e0b/src/sampling.jl#L445-L456

In our code, we use a simple sample fucntion, which has a stucture below
## sample([rng], a, [wv::AbstractWeights], n::Integer; replace=true, ordered=false) ##

For weight factors [wv::AbstractWeights], please see https://juliastats.org/StatsBase.jl/stable/weights/
There are different weight vectors. the type of weight in our code is Probability weights.
(https://juliastats.org/StatsBase.jl/stable/weights/#ProbabilityWeights-1)
Attention!
U need delcare you weight type in the sample function or the sample function will not take it as a valid input
##
example:(this is invalid) ✘ the right one is in our code.
sample(Grid_Label_flatten::AbstractArray,✘final_probability_flatten✘, amount_panel , replace = false, ordered= true)
##
"""

function Make_Question(n::Int,m::Int,factor::Int = 10, Density::Float64 = 0.75, option::String = "1-norm", distance::Int = 2)
    Grid_Label =  transpose( reshape(  collect(1:n*m),(n,m) ) )
    Grid_Label_flatten = reshape(Grid_Label,(m*n))
    
    px = abs.(((Grid_Label.-1) .% n).-((n-1)/2) )
    py = abs.(((a_python.-1) .÷ n).-((n-1)/2))
    mesh_porbability = 2 .^ -(px.+py)
    final_probability = mesh_porbability / sum(mesh_porbability)
    final_probability_flatten= reshape(final_probability, (m*n))
    
    amount_panel = Int(round(Density*n*m))
    
    allocation = sample(Grid_Label_flatten::AbstractArray, ProbabilityWeights(final_probability_flatten), amount_panel , replace = false, ordered= true)
    
    return allocation
end

Make_Question (generic function with 5 methods)

In [387]:
n = 6
m = 7
Grid_Label =  transpose( reshape(  collect(1:n*m),(n,m) ) )


7×6 Transpose{Int64,Array{Int64,2}}:
  1   2   3   4   5   6
  7   8   9  10  11  12
 13  14  15  16  17  18
 19  20  21  22  23  24
 25  26  27  28  29  30
 31  32  33  34  35  36
 37  38  39  40  41  42

In [388]:
answer = Make_Question(n::Int,m::Int);
reshape(answer,(4,8))

4×8 Array{Int64,2}:
 1   7  11  15  19  23  28  32
 3   8  12  16  20  25  29  33
 4   9  13  17  21  26  30  34
 5  10  14  18  22  27  31  41

In [389]:
size(answer)

(32,)

In [401]:
position_x = answer.÷ n

32-element Array{Int64,1}:
 0
 0
 0
 0
 1
 1
 1
 1
 1
 2
 2
 2
 2
 ⋮
 3
 4
 4
 4
 4
 4
 5
 5
 5
 5
 5
 6

In [402]:
position_y = answer.% n

32-element Array{Int64,1}:
 1
 3
 4
 5
 1
 2
 3
 4
 5
 0
 1
 2
 3
 ⋮
 5
 1
 2
 3
 4
 5
 0
 1
 2
 3
 4
 5

In [403]:
size(position_x)

(32,)

In [404]:
k = transpose(hcat(position_x,position_y))

2×32 Transpose{Int64,Array{Int64,2}}:
 0  0  0  0  1  1  1  1  1  2  2  2  2  …  3  4  4  4  4  4  5  5  5  5  5  6
 1  3  4  5  1  2  3  4  5  0  1  2  3     5  1  2  3  4  5  0  1  2  3  4  5

In [405]:
position = reshape( k, (1 ,2 ,size(position_x)[1]))

1×2×32 reshape(::Transpose{Int64,Array{Int64,2}}, 1, 2, 32) with eltype Int64:
[:, :, 1] =
 0  1

[:, :, 2] =
 0  3

[:, :, 3] =
 0  4

...

[:, :, 30] =
 5  3

[:, :, 31] =
 5  4

[:, :, 32] =
 6  5

In [406]:
println(position[:,:,1])
norm(position[:,:,1],1)

[0 1]


1.0

In [407]:
a = position .- position[:,:,1]

1×2×32 Array{Int64,3}:
[:, :, 1] =
 0  0

[:, :, 2] =
 0  2

[:, :, 3] =
 0  3

...

[:, :, 30] =
 5  2

[:, :, 31] =
 5  3

[:, :, 32] =
 6  4

In [408]:
b = [norm(slice,2) for slice in eachslice(a, dims=3)]

32-element Array{Float64,1}:
 0.0
 2.0
 3.0
 4.0
 1.0
 1.4142135623730951
 2.23606797749979
 3.1622776601683795
 4.123105625617661
 2.23606797749979
 2.0
 2.23606797749979
 2.8284271247461903
 ⋮
 5.0
 4.0
 4.123105625617661
 4.47213595499958
 5.0
 5.656854249492381
 5.0990195135927845
 5.0
 5.0990195135927845
 5.385164807134504
 5.830951894845301
 7.211102550927978

In [409]:
cond = b .<= 1

32-element BitArray{1}:
 1
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [410]:
index = findall(x->x == 1, cond)

2-element Array{Int64,1}:
 1
 5

In [416]:
final_index = answer[index]

2-element Array{Int64,1}:
 1
 7

In [421]:
map = zeros((n*m,n*m))
map[1 , [i for i in final_index] ] .= 1
map

42×42 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [279]:
 transpose( reshape(  collect(1:n*m),(n,m) ) )

7×6 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
  1   2   3   4   5   6
  7   8   9  10  11  12
 13  14  15  16  17  18
 19  20  21  22  23  24
 25  26  27  28  29  30
 31  32  33  34  35  36
 37  38  39  40  41  42